<a href="https://colab.research.google.com/github/ithingv/nlp_study/blob/main/preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#전처리 과정

- 코퍼스 수집
    1. 공개된 데이터(각종 문제 또는 논문을 위한 데이터, 양이 한정적)
    2. 크롤링을 통한 데이터 수집
        - 웹 서버 리소스 사용 가능 유무는 robot.txt를 보고 판단
        - selnium, bs4, phantomJS, headless brower
        - 자동 기계번역을 위한 병렬 코퍼스는 단일 코퍼스에 비해 상당히 어렵다.
            - Korean Parallel Data
            - https://sites.google.com/site/koreanparalleldata/
- 정제
    - 전각문자제거
        -> 반각문자로 변환
    - 대소문자 통일
        - 표현을 일원화하여 여러 단어를 하나의 형태로 통일해 희소성을 줄이는 효과
        - 단어 임베딩을 통한 효율적인 표현이 가능해지므로 대소문자 통일의 필요성이 줄어들었다.
    - 정규표현식 https://regexper.com
    - 치환자
- 문장 단위 분절
    - 텍스트 입력이 문장 단위인 경우가 대부분이다.
    - 여러 문장이 한 라인에 있거나, 한 문장이 여러 라인에 걸쳐 있는 경우(문장 합치기)에는 문장 단위 분절이 필요하다.
- 분절
    - 한국어(Mecab), 일본어(Mecab), 중국어(Jieba)
    - 일반적이고 전형적인 쉬운 문장을 처리하는 능력은 비슷
    - 고유명사나 신조어를 처리하는 능력은 라이브러리 별로 차이가 있으므로 비교를 잘 해야함
- 병렬 코퍼스 정렬(생략가능)
- 서브워드 분절
    - BPE 알고리즘을 통한 서브워드 단위 분절은 현재 필수 전처리 방법으로 자리잡았다. 서브워드 분절은 기본적으로 '단어는 의미를 가진 더 작은 서브워드들의 조합으로 이루어진다'는 가정하게 적용되는 알고리즘이다.
    
    - 영어나 한국어 역시 라틴어와 한자를 기반으로 형성된 언어이기에 많은 단어가 서브워드들로 구성된다. 따라서 적절한 서브워드를 발견하여 해당 단위로 쪼개어주면 어휘 수를 줄일 수 있고 **희소성**을 효과적으로 줄일 수 있다.

    - ex) concentrate ---> con(=together) + centr(=center) + ate(=make)
    - ex) 집중 ---> (모을)집 + (가운데)중

    - 서브워드 단위 분절로 얻는 가장 대표적인 효과는 UNK 토큰에 대한 효율적인 대처이다. 
    - 대부분의 딥러닝 NLP 알고리즘은 입력 문장을 단어들의 시퀀스로 받아들인다. UNK 토큰이 나타나면 이후의 언어 모델의 확률은 크게 망가지고 적절한 문장의 임베딩 또는 생성이 어렵게된다. (이전 단어가 다음 단어가 출현 확률에 영향을 미침)

    - 서브워드 단위 분절을 통해 신조어나 오타와 같은 UNK에 대해 서브워드 단위나 문자 단위로 쪼개줌으로써 기존 훈련 데이터에서 보았던 토큰들의 조합으로 만들어버릴 수 있다.
    - 즉 UNK 자체를 없앰으로써 효율적으로 UNK에 대처할 수 있다.
    - BPE를 적욯할 경우 원래 문장의 띄어쓰기와 BPE로 인한 띄어쓰기를 서로 구분하기 위해 기존 단어 앞에 특수 문자 '_'를 사용한다.(detokenize 할 때 필요)
    - 오픈소스
        - Sennrich: https://github.com/rsennrich/subword-nmt
        - 수정버전: https://github.com/kh-kim/subword-nmt
        - SentencePiece: https://github.com/google/sentencepiece
        


In [8]:
# 문장단위분절 예제

import sys, fileinput, re
import nltk
from nltk.tokenize import sent_tokenize

lines = "자연어처리는 인공지능의 한 줄기 입니다. 시퀀스 투 시퀀스의 등장 이후로 딥러닝을 활용한 자연어처리는 새로운 전기를 맞이하게 되었습니다. 문장을 받아 단순히 수치로 나타내던 시절을 넘어, 원하는대로 문장을 만들어낼 수 있게 된 것입니다."
nltk.download('punkt')

if __name__ == "__main__":
    for line in lines:
        if line.strip() != "":
            line = re.sub(r'([a-z])\.([A-Z])', r'\1. \2', line.strip())

            sentences = sent_tokenize(line.strip())

            for s in sentences:
                if s != "":
                    sys.stdout.write(s + '\n')

# Torchtext

- 자연어 처리 문제 또는 텍스트에 관한 머신러닝이나 딥러닝을 수행하는 데이터를 읽고 전처리하는 코드를 모아둔 라이브러리이다. 
- 예시

|x데이터|y데이터| 활용분야|
|:-----:|:----:|:---:|
| 코퍼스   | 클래스|텍스트 분류, 감성분석|
| 코퍼스   | - |언어모델|
| 코퍼스   | 코퍼스 | 기계번역, 요약, QuestionAnswering |



In [5]:
# 코퍼스와 레이블 읽기
# 클래스와 텍스트가 탭('\t')로 구분된 데이터의 입력을 받는 내용
# dataloader.py

from torchtext import data

class DataLoader(object):
    def __init__(self, train_fn, valid_fn, 
                 batch_size=64,
                 device=-1,
                 max_vocab=999999,
                 min_freq=1,
                 use_eos=False,
                 shuffle=True
                 ):
        
        super(DataLoader, self).__init__()

        # Define field of the input file
        # The input file consists of two fields
        # https://torchtext.readthedocs.io/en/latest/data.html
        
        self.label = data.Field(sequential=False, # Whether the datatype represents sequential data. If False, no tokenization is applied. Default: True.
                                use_vocab=True, # Whether to use a Vocab object. If False, the data in this field should already be numerical. Default: True.
                                unk_token=None # unk_token – The string token used to represent OOV words. Default: “<unk>”.
                                )
        
        self.text = data.Field(use_vocab=True,
                        batch_first=True,
                        include_lengths=False, # Whether to return a tuple of a padded minibatch and a list containing the lengths of each examples, or just a padded minibatch. Default: False.
                        eos_token='<EOS>' if use_eos else None # A token that will be appended to every example using this field, or None for no end-of-sentence token. Default: None.
                        )

        train, valid = data.TabularDataset.splits(path='',
                                                  train=train_fn,
                                                  validation=valid_fn,
                                                  format='tsv',
                                                  fields=[('label', self.label),
                                                          ('text', self.text)
                                                          ]
                                                )
        
        # Those loaded dataset would be feeded into each iterator:
        # train iterator and valid iterator
        # We sort input sentences by length, to group similar lengths
        self.train_iter, self.valid_iter = data.BucketIterator.splits((train, valid),
                                                                      batch_size=batch_size,
                                                                      device='cuda:%d' % device if device >=0 else 'cpu',
                                                                      shuffle=shuffle,
                                                                      sort_key=lambda x: len(x.text),
                                                                      sort_within_batch=True
        )

        # At last, we make a vocabulary for label and text field
        # It is making mapping table between words and indice
        self.label.build_vocab(train)
        self.text_build_vocab(train, max_size=max_vocab, min_freq=min_freq)

# 코퍼스 읽기
    한 라인이 텍스트로만 채워져 있을 때를 위한 코드
    주로 언어 모델을 훈련시키는 상황에 쓸 수 있다.
    LanguageModel Dataset을 통해 미리 정의된 필드를 텍스트 파일에서 읽어들인다.
    이때 각 문장의 길이에 따라 정렬을 통해 비슷한 길이의 문장끼리 미니배치를 만들어준다.
    이 작업을 통해 매우 상이한 길이의 문장들이 하나의 미니배치에 묶여 훈련 시간에서 손해보는 것을 방지한다.

In [ ]:
# 코퍼스와 레이블 읽기
# 클래스와 텍스트가 탭('\t')로 구분된 데이터의 입력을 받는 내용
# dataloader.py

from torchtext import data, datasets

PAD, BOS, EOS = 1, 2, 3

class DataLoader(object):
    def __init__(self, 
                 train_fn, 
                 valid_fn, 
                 batch_size=64,
                 device='cpu',
                 max_vocab=999999,
                 max_length=255,
                 fix_length=None,
                 use_bos=True,
                 use_eos=True,
                 shuffle=True
                 ):
        
        super(DataLoader, self).__init__()

        # Define field of the input file
        # The input file consists of two fields
        # https://torchtext.readthedocs.io/en/latest/data.html
        
        
        self.text = data.Field(sequential=True,
                                use_vocab=True,
                                batch_first=True,
                                include_lengths=True, # Whether to return a tuple of a padded minibatch and a list containing the lengths of each examples, or just a padded minibatch. Default: False.
                                fix_length=fix_length,
                                init_token='<BOS>' if use_bos else None,
                                eos_token='<EOS>' if use_eos else None # A token that will be appended to every example using this field, or None for no end-of-sentence token. Default: None.
                                )

        train = LanguageModelDataset(path=train_fn,
                                     fields=[('text', self.text)],
                                     max_length=max_length
        )     
        
        train = LanguageModelDataset(path=valid_fn,
                                     fields=[('text', self.text)],
                                     max_length=max_length
        )     

        self.train_iter, self.valid_iter = data.BucketIterator(train,
                                                               batch_size=batch_size,
                                                               device='cuda:%d' % device if device >=0 else 'cpu',
                                                               shuffle=shuffle,
                                                               sort_key=lambda x: -len(x.text),
                                                                      sort_within_batch=True
        )

        # At last, we make a vocabulary for label and text field
        # It is making mapping table between words and indice
        self.label.build_vocab(train)
        self.text_build_vocab(train, max_size=max_vocab, min_freq=min_freq)



class LanguageModelDataset(data.Dataset):
    def __init__(self, path, fields, max_length=None, **kwargs):
        if not isinstance(fields[0], (tuple, list)):
            fields = [('text', fields[0])]
        
        examples = []
        with open(path) as  f:
            for line in f:
                line = line.strip()
                if max_length and max_length < len(line.split()):
                    continue
                if line != '':
                    examples.append(data.Example.fromlist(
                        [line], fields))

        super(LanguageModelDataset, self).__init__(examples, fields, **kwargs)